In [1]:
import torch
import os
import skimage
from PIL import Image
from torchvision.transforms import Resize, Compose, ToTensor, Normalize
from pathlib import Path
from torch.utils.data import DataLoader, Dataset
from training.wandblogger import WandBLogger2D
from training.trainer import MRTrainer
from datasets.imagesignal import ImageSignal
from networks.mrnet import MRFactory
from datasets.imageutils import gaussian_pyramid2D, gaussian_tower2D, box_kernel2D
import yaml
from yaml.loader import SafeLoader

In [2]:
os.environ["WANDB_NOTEBOOK_NAME"] = "m-net.ipynb"
BASE_DIR = Path('.').absolute().parents[0]
IMAGE_PATH = BASE_DIR.joinpath('img')

In [7]:
project_name = "testing_dictionary_test"
# hyperparameters
# hyperparameters
with open('../configs/config_base_m_net.yml') as f:
    hyper = yaml.load(f, Loader=SafeLoader)
    print(hyper)


{'omega_0': [4, 8, 16, 32, 64, 128], 'in_features': 2, 'hidden_features': '-48 -64 -64 -128 -128 -256', 'hidden_layers': 1, 'superposition_w0': False, 'hidden_omega_0': '-30 -40 -50 -60 -70 -80', 'sampling_scheme': 'uniform', 'multiresolution': 'pyramid', 'max_epochs_per_stage': '-500 -450 -350 -250 -150 -100', 'opt_method': 'Adam', 'loss_function': 'd0_MSE', 'lr': 0.0001, 'loss_tol': 1e-14, 'diff_tol': 1e-11, 'batch_size': 16641, 'image_name': 'lena257.png', 'width': 129, 'height': 129, 'channels': 1, 'stage': 1, 'max_stages': 6, 'model': 'M', 'useattributes': True, 'device': 'cuda', 'eval_device': 'cuda'}


In [4]:
BATCH_SIZE = hyper['batch_size']
kernel = box_kernel2D(5)

In [5]:

base_signal = ImageSignal.init_fromfile(
                    os.path.join(IMAGE_PATH, hyper['image_name']),
                    useattributes=hyper.get('useattributes', False),
                    batch_pixels=hyper['batch_size'],
                    width=hyper['width'],height= hyper['height'])
if hyper['multiresolution'] == 'capacity':
    train_dataloader = DataLoader(base_signal, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=0)
    test_dataloader = DataLoader(base_signal, batch_size=BATCH_SIZE, pin_memory=True, num_workers=0)
else:
    pyramid = gaussian_pyramid2D(base_signal, hyper['max_stages'], kernel)
    tower = gaussian_tower2D(base_signal, hyper['max_stages'], kernel)
    trainsource = pyramid if hyper['multiresolution'] == 'pyramid' else tower
    train_dataloader = [DataLoader(signal, shuffle=True, batch_size=BATCH_SIZE) 
                        for signal in trainsource]
    test_dataloader = [DataLoader(signal, batch_size=BATCH_SIZE) 
                        for signal in tower]

In [6]:
wandblogger = WandBLogger2D(project_name,
                            f"{hyper['model']}{hyper['multiresolution'][0].upper()}{hyper['image_name'][0:4]}_",
                            hyper,
                            BASE_DIR)
mrmodel = MRFactory.from_dict(hyper)
print("Model: ", type(mrmodel))
mrtrainer = MRTrainer.init_from_dict(mrmodel, train_dataloader, test_dataloader, wandblogger, hyper)
mrtrainer.train(hyper['device'])

TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
